In [7]:
import keras 
import numpy as np
import pandas as pd
import os
import sklearn
import xgboost as xgb
import operator
import optuna


from functools import partial
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from matplotlib import pylab as plt
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

# import warnings
# warnings.filterwarnings('ignore')
# def fxn():
#     warnings.warn("deprecated", DeprecationWarning)

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     fxn()

In [9]:
#csvデータの呼び出し場所
loadpath = "../input/"
#csvデータの保存場所
savepath = "../output/"

In [49]:
#データ呼び出し
train = pd.read_csv(loadpath + "train.csv")
test = pd.read_csv(loadpath + "test.csv")
train.set_index('id',inplace=True)
test.set_index('id',inplace=True)

labels = train.target
# labels = pd.get_dummies(labels,prefix='Survived')

train.drop('target', axis=1, inplace=True)

In [65]:
def baseline_model():
    model = Sequential()
    model.add(Dense(64, activation='relu',input_dim=train.shape[1]))
    model.add(keras.layers.core.Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2,activation='softmax'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [66]:
model = KerasClassifier(build_fn=baseline_model, epochs=100)

In [67]:
model.fit(train, labels, batch_size=100, verbose=True)

Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 0.9832 - acc: 0.490 - 1s 2ms/step - loss: 0.9299 - acc: 0.5040
Epoch 2/100
250/250 [==============================] - ETA: 0s - loss: 0.7114 - acc: 0.600 - 0s 32us/step - loss: 0.6912 - acc: 0.6160
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.6885 - acc: 0.610 - 0s 37us/step - loss: 0.6628 - acc: 0.6680
Epoch 4/100
250/250 [==============================] - ETA: 0s - loss: 0.5673 - acc: 0.720 - 0s 47us/step - loss: 0.5995 - acc: 0.7160
Epoch 5/100
250/250 [==============================] - ETA: 0s - loss: 0.5642 - acc: 0.720 - 0s 36us/step - loss: 0.5830 - acc: 0.7000
Epoch 6/100
250/250 [==============================] - ETA: 0s - loss: 0.6072 - acc: 0.690 - 0s 33us/step - loss: 0.5495 - acc: 0.7280
Epoch 7/100
250/250 [==============================] - ETA: 0s - loss: 0.5504 - acc: 0.700 - 0s 30us/step - loss: 0.5026 - acc: 0.7440
Epoch 8/100
250/250 [==============================] - E

250/250 [==============================] - ETA: 0s - loss: 0.0184 - acc: 0.990 - 0s 72us/step - loss: 0.0170 - acc: 0.9920
Epoch 62/100
250/250 [==============================] - ETA: 0s - loss: 0.0093 - acc: 1.000 - 0s 31us/step - loss: 0.0142 - acc: 1.0000
Epoch 63/100
250/250 [==============================] - ETA: 0s - loss: 0.0142 - acc: 1.000 - 0s 30us/step - loss: 0.0099 - acc: 1.0000
Epoch 64/100
250/250 [==============================] - ETA: 0s - loss: 0.0084 - acc: 1.000 - 0s 31us/step - loss: 0.0169 - acc: 0.9960
Epoch 65/100
250/250 [==============================] - ETA: 0s - loss: 0.0101 - acc: 1.000 - 0s 30us/step - loss: 0.0142 - acc: 1.0000
Epoch 66/100
250/250 [==============================] - ETA: 0s - loss: 0.0124 - acc: 1.000 - 0s 31us/step - loss: 0.0104 - acc: 1.0000
Epoch 67/100
250/250 [==============================] - ETA: 0s - loss: 0.0088 - acc: 1.000 - 0s 30us/step - loss: 0.0158 - acc: 0.9920
Epoch 68/100
250/250 [==============================] - ETA: 

In [80]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
scores = cross_validate(model, X=train, y=labels, cv=kf, scoring='roc_auc')

Epoch 1/100
200/200 [==============================] - ETA: 4s - loss: 0.6942 - acc: 0.625 - 1s 4ms/step - loss: 0.8336 - acc: 0.5650
Epoch 2/100
200/200 [==============================] - ETA: 0s - loss: 0.4934 - acc: 0.687 - 0s 98us/step - loss: 0.6160 - acc: 0.6600
Epoch 3/100
200/200 [==============================] - ETA: 0s - loss: 0.4508 - acc: 0.812 - 0s 86us/step - loss: 0.5772 - acc: 0.7000
Epoch 4/100
200/200 [==============================] - ETA: 0s - loss: 0.5161 - acc: 0.687 - 0s 87us/step - loss: 0.5262 - acc: 0.6950
Epoch 5/100
200/200 [==============================] - ETA: 0s - loss: 0.4343 - acc: 0.781 - 0s 79us/step - loss: 0.4664 - acc: 0.7850
Epoch 6/100
200/200 [==============================] - ETA: 0s - loss: 0.3769 - acc: 0.843 - 0s 76us/step - loss: 0.4200 - acc: 0.7800
Epoch 7/100
200/200 [==============================] - ETA: 0s - loss: 0.4311 - acc: 0.843 - 0s 76us/step - loss: 0.3871 - acc: 0.8350
Epoch 8/100
200/200 [==============================] - E

200/200 [==============================] - ETA: 0s - loss: 2.0531e-04 - acc: 1.000 - 0s 77us/step - loss: 0.0028 - acc: 1.0000
Epoch 62/100
200/200 [==============================] - ETA: 0s - loss: 0.0016 - acc: 1.000 - 0s 84us/step - loss: 0.0024 - acc: 1.0000
Epoch 63/100
200/200 [==============================] - ETA: 0s - loss: 0.0031 - acc: 1.000 - 0s 76us/step - loss: 0.0055 - acc: 0.9950
Epoch 64/100
200/200 [==============================] - ETA: 0s - loss: 0.0080 - acc: 1.000 - 0s 83us/step - loss: 0.0015 - acc: 1.0000
Epoch 65/100
200/200 [==============================] - ETA: 0s - loss: 2.0725e-04 - acc: 1.000 - 0s 81us/step - loss: 0.0122 - acc: 0.9950
Epoch 66/100
200/200 [==============================] - ETA: 0s - loss: 7.5609e-05 - acc: 1.000 - 0s 80us/step - loss: 0.0381 - acc: 0.9900
Epoch 67/100
200/200 [==============================] - ETA: 0s - loss: 6.5137e-04 - acc: 1.000 - 0s 76us/step - loss: 3.1368e-04 - acc: 1.0000
Epoch 68/100
200/200 [===================

Epoch 20/100
200/200 [==============================] - ETA: 0s - loss: 0.1091 - acc: 0.968 - 0s 64us/step - loss: 0.0615 - acc: 0.9900
Epoch 21/100
200/200 [==============================] - ETA: 0s - loss: 0.0697 - acc: 0.968 - 0s 66us/step - loss: 0.0491 - acc: 0.9900
Epoch 22/100
200/200 [==============================] - ETA: 0s - loss: 0.0197 - acc: 1.000 - 0s 64us/step - loss: 0.0511 - acc: 0.9800
Epoch 23/100
200/200 [==============================] - ETA: 0s - loss: 0.0699 - acc: 0.937 - 0s 64us/step - loss: 0.0461 - acc: 0.9750
Epoch 24/100
200/200 [==============================] - ETA: 0s - loss: 0.0865 - acc: 0.968 - 0s 65us/step - loss: 0.0640 - acc: 0.9800
Epoch 25/100
200/200 [==============================] - ETA: 0s - loss: 0.0240 - acc: 1.000 - 0s 64us/step - loss: 0.0275 - acc: 0.9950
Epoch 26/100
200/200 [==============================] - ETA: 0s - loss: 0.0231 - acc: 1.000 - 0s 64us/step - loss: 0.0390 - acc: 0.9900
Epoch 27/100
200/200 [==========================

Epoch 80/100
200/200 [==============================] - ETA: 0s - loss: 0.0844 - acc: 0.968 - 0s 65us/step - loss: 0.0192 - acc: 0.9900
Epoch 81/100
200/200 [==============================] - ETA: 0s - loss: 1.9950e-04 - acc: 1.000 - 0s 65us/step - loss: 3.9488e-04 - acc: 1.0000
Epoch 82/100
200/200 [==============================] - ETA: 0s - loss: 6.8231e-06 - acc: 1.000 - 0s 64us/step - loss: 0.0018 - acc: 1.0000
Epoch 83/100
200/200 [==============================] - ETA: 0s - loss: 1.1171e-05 - acc: 1.000 - 0s 65us/step - loss: 0.0014 - acc: 1.0000
Epoch 84/100
200/200 [==============================] - ETA: 0s - loss: 2.2556e-04 - acc: 1.000 - 0s 63us/step - loss: 3.6654e-04 - acc: 1.0000
Epoch 85/100
200/200 [==============================] - ETA: 0s - loss: 4.1644e-05 - acc: 1.000 - 0s 67us/step - loss: 6.2923e-05 - acc: 1.0000
Epoch 86/100
200/200 [==============================] - ETA: 0s - loss: 1.2771e-05 - acc: 1.000 - 0s 64us/step - loss: 5.6494e-04 - acc: 1.0000
Epoch 87

200/200 [==============================] - ETA: 0s - loss: 0.0569 - acc: 0.968 - 0s 66us/step - loss: 0.0183 - acc: 0.9950
Epoch 40/100
200/200 [==============================] - ETA: 0s - loss: 0.0217 - acc: 1.000 - 0s 64us/step - loss: 0.0170 - acc: 1.0000
Epoch 41/100
200/200 [==============================] - ETA: 0s - loss: 8.9929e-04 - acc: 1.000 - 0s 64us/step - loss: 0.0124 - acc: 0.9950
Epoch 42/100
200/200 [==============================] - ETA: 0s - loss: 0.0018 - acc: 1.000 - 0s 65us/step - loss: 0.0082 - acc: 1.0000
Epoch 43/100
200/200 [==============================] - ETA: 0s - loss: 0.0135 - acc: 1.000 - 0s 64us/step - loss: 0.0089 - acc: 1.0000
Epoch 44/100
200/200 [==============================] - ETA: 0s - loss: 0.0033 - acc: 1.000 - 0s 66us/step - loss: 0.0130 - acc: 0.9950
Epoch 45/100
200/200 [==============================] - ETA: 0s - loss: 0.0252 - acc: 1.000 - 0s 66us/step - loss: 0.0331 - acc: 0.9900
Epoch 46/100
200/200 [==============================] - E

Epoch 98/100
200/200 [==============================] - ETA: 0s - loss: 0.0041 - acc: 1.000 - 0s 76us/step - loss: 8.4880e-04 - acc: 1.0000
Epoch 99/100
200/200 [==============================] - ETA: 0s - loss: 1.3910e-05 - acc: 1.000 - 0s 75us/step - loss: 2.9861e-04 - acc: 1.0000
Epoch 100/100
200/200 [==============================] - ETA: 0s - loss: 2.8312e-07 - acc: 1.000 - 0s 75us/step - loss: 4.7428e-04 - acc: 1.0000
Epoch 1/100
200/200 [==============================] - ETA: 4s - loss: 1.5710 - acc: 0.250 - 1s 4ms/step - loss: 0.9503 - acc: 0.4750
Epoch 2/100
200/200 [==============================] - ETA: 0s - loss: 0.8073 - acc: 0.468 - 0s 77us/step - loss: 0.7008 - acc: 0.5950
Epoch 3/100
200/200 [==============================] - ETA: 0s - loss: 0.5994 - acc: 0.687 - 0s 76us/step - loss: 0.5865 - acc: 0.6950
Epoch 4/100
200/200 [==============================] - ETA: 0s - loss: 0.4342 - acc: 0.812 - 0s 74us/step - loss: 0.5320 - acc: 0.7200
Epoch 5/100
200/200 [===========

200/200 [==============================] - ETA: 0s - loss: 3.6247e-04 - acc: 1.000 - 0s 76us/step - loss: 5.2510e-04 - acc: 1.0000
Epoch 59/100
200/200 [==============================] - ETA: 0s - loss: 0.0027 - acc: 1.000 - 0s 78us/step - loss: 0.0043 - acc: 1.0000
Epoch 60/100
200/200 [==============================] - ETA: 0s - loss: 0.0021 - acc: 1.000 - 0s 75us/step - loss: 0.0019 - acc: 1.0000
Epoch 61/100
200/200 [==============================] - ETA: 0s - loss: 1.5404e-04 - acc: 1.000 - 0s 75us/step - loss: 0.0039 - acc: 1.0000
Epoch 62/100
200/200 [==============================] - ETA: 0s - loss: 0.0024 - acc: 1.000 - 0s 77us/step - loss: 0.0030 - acc: 1.0000
Epoch 63/100
200/200 [==============================] - ETA: 0s - loss: 0.0046 - acc: 1.000 - 0s 76us/step - loss: 0.0286 - acc: 0.9900
Epoch 64/100
200/200 [==============================] - ETA: 0s - loss: 0.0028 - acc: 1.000 - 0s 75us/step - loss: 0.0013 - acc: 1.0000
Epoch 65/100
200/200 [===========================

Epoch 17/100
200/200 [==============================] - ETA: 0s - loss: 0.1131 - acc: 0.968 - 0s 75us/step - loss: 0.1154 - acc: 0.9700
Epoch 18/100
200/200 [==============================] - ETA: 0s - loss: 0.0668 - acc: 1.000 - 0s 78us/step - loss: 0.1044 - acc: 0.9700
Epoch 19/100
200/200 [==============================] - ETA: 0s - loss: 0.0850 - acc: 0.968 - 0s 75us/step - loss: 0.0875 - acc: 0.9750
Epoch 20/100
200/200 [==============================] - ETA: 0s - loss: 0.0376 - acc: 1.000 - 0s 77us/step - loss: 0.0669 - acc: 0.9850
Epoch 21/100
200/200 [==============================] - ETA: 0s - loss: 0.0519 - acc: 1.000 - 0s 78us/step - loss: 0.0564 - acc: 0.9950
Epoch 22/100
200/200 [==============================] - ETA: 0s - loss: 0.1267 - acc: 0.937 - 0s 83us/step - loss: 0.0612 - acc: 0.9850
Epoch 23/100
200/200 [==============================] - ETA: 0s - loss: 0.0216 - acc: 1.000 - 0s 75us/step - loss: 0.0518 - acc: 0.9850
Epoch 24/100
200/200 [==========================

200/200 [==============================] - ETA: 0s - loss: 7.0553e-04 - acc: 1.000 - 0s 79us/step - loss: 0.0011 - acc: 1.0000
Epoch 77/100
200/200 [==============================] - ETA: 0s - loss: 1.2407e-04 - acc: 1.000 - 0s 77us/step - loss: 5.0751e-04 - acc: 1.0000
Epoch 78/100
200/200 [==============================] - ETA: 0s - loss: 2.3271e-04 - acc: 1.000 - 0s 77us/step - loss: 4.0401e-04 - acc: 1.0000
Epoch 79/100
200/200 [==============================] - ETA: 0s - loss: 0.0013 - acc: 1.000 - 0s 76us/step - loss: 0.0078 - acc: 0.9950
Epoch 80/100
200/200 [==============================] - ETA: 0s - loss: 1.0353e-04 - acc: 1.000 - 0s 77us/step - loss: 3.8871e-04 - acc: 1.0000
Epoch 81/100
200/200 [==============================] - ETA: 0s - loss: 5.2271e-05 - acc: 1.000 - 0s 75us/step - loss: 6.4316e-04 - acc: 1.0000
Epoch 82/100
200/200 [==============================] - ETA: 0s - loss: 9.7230e-07 - acc: 1.000 - 0s 76us/step - loss: 6.4210e-04 - acc: 1.0000
Epoch 83/100
200/

In [81]:
scores['test_score']

array([0.62326389, 0.76215278, 0.71354167, 0.64236111, 0.66145833])

In [72]:
pred = model.predict(test)

In [79]:
# create submission file
submission_result = pd.DataFrame({"target":pred},index=test.index)
submission_result.to_csv(savepath + 'submission_dnn.csv',index='False')

In [77]:
submission_result

,target
id,
250,1.0
251,0.0
252,0.0
253,1.0
254,1.0
255,0.0
256,0.0
257,1.0
258,1.0
